In [2]:
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt

from sklearn import datasets

import nltk
import re

In [3]:
nltk.download("averaged_perceptron_tagger")
nltk.download("maxent_ne_chunker")
nltk.download("words")

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Adam\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\Adam\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Adam\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [5]:
def sent_to_tree(sentence):
    tokens = nltk.word_tokenize(sentence)
    tagged = nltk.tag.pos_tag(tokens, )
    tree = nltk.chunk.ne_chunk(tagged)
    
    return tree

In [6]:
def tree_to_df(tree):
    flat_tree = tree.flatten()[:]
    df = pd.DataFrame(flat_tree, columns=["word", "pos"])
    
    return df

In [61]:
def url_clean(s):
    return re.sub("http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", "", s)

def junk_clean(s):
    # Any non-(letter, space) characters
    return re.sub("[^a-zA-z\s]", "", s)

In [67]:
def word_count(s):
    junk_regex = "[^a-zA-z\s]"
    
    s = url_clean(s.lower())
    sent_df = tree_to_df(sent_to_tree(s))

    
    cond = (~ sent_df["word"].str.match(junk_regex)) & sent_df["pos"].isin(topic_pos) & (sent_df["word"].str.len() > 1)
    return sent_df.loc[cond, "word"].value_counts()

---

In [8]:
pos_tags = pd.read_clipboard()
pos_tags

,Tag,Description,Example
0,CC,"conjunction, coordinating","and, or, but"
1,CD,cardinal number,"five, three, 13%"
2,DT,determiner,"the, a, these"
3,EX,existential there,there were six boys
4,FW,foreign word,mais
5,IN,"conjunction, subordinating or preposition","of, on, before, unless"
6,JJ,adjective,"nice, easy"
7,JJR,"adjective, comparative","nicer, easier"
8,JJS,"adjective, superlative","nicest, easiest"
9,LS,list item marker,


In [42]:
important_pos = ["CD", "EX", "IN", ""] #...
unimportant_pos = ["CC", "FW", "LS", "POS", "SYM", "UH",] + list(",:()")
topic_pos = ["JJ", "VBG", "JJ", "NNS", "NNP", "JJS", "NN"]

---

### Tweets

In [72]:
file = "C:\\Users\\Adam\\PycharmProjects\\NLP\\datasets\\trump_tweets.txt"
with open(file, encoding="utf-8") as f:
    tweets = f.read()

In [9]:
sent_df = tree_to_df(sent_to_tree(sentence[:10000].lower()))

---

### Article

In [65]:
article1 = '''

Minister of State for European Affairs Helen McEntee says it is time for the House of Commons “to figure out what it wants”.

“We know what they don’t want, we’ve known that for over two years,” she told Newstalk’s Pat Kenny show.

Ms McEntee said that she hopes there will not be contradictory votes in the House of Commons this week. MPs need to listen to their own experts and business groups “and take that on board when they’re voting.”

British prime minister Theresa May’s fragile authority suffered another blow on Monday as three ministers quit to back a Commons amendment enabling MPs to take control of Commons business to stage a series of “indicative votes” on alternatives to her deal.

Ms McEntee said that preparations are well under way for Brexit, but warned that there was only so much that could be done until a definite outcome is known. “It takes time to adapt and change.”

Even if there is an organised Brexit it will still mean changes, she said.

“Even in a best case scenario there will be changes, people will have to adapt.”

This will force businesses to look at their markets and make alternative arrangements rather than be so reliant on one market.

“There’s something positive in that, but not the positive we want.”

While discussions are ongoing about the levels of support that will be made available if necessary, Ms McEntee said that exact figures are not yet known. “It is very clear that support will be there.”

When asked why Ireland had not benefitted more from companies leaving the UK, she pointed out that “Ireland got its share.”

There might be some boost to the economy from such moves, but not enough. “There needs to be a complete restructuring of our economy.

“Either way this is not going to be good for our economy.

“I’m hopeful that we can make this work.”

The Cabinet is likely to discuss Brexit preparations at its meeting on Tuesday morning, while Mr Varadkar is expected to face questions on the subject when the Dáil meets for the first time after the St Patrick’s Day recess.

Senior Government sources have said intensive planning is taking place with the European Commission on how to avoid a hard border in Ireland in the event of a no-deal Brexit. However, the officials remain tight-lipped about how this can be achieved.

'''

article2 = '''


Minister for Health Simon Harris has defended the Government’s commitment to adopting Sláintecare, contending the plan has “come alive” in recent weeks.

Mr Harris also rejected a claim the Government has allotted only €20 million in 2019 to the ambitious blueprint to change the health services from a two-tier, public-private, service to a single-tier service over 10 years.

“Over €200 million will be spent on Sláintecare this year arising from announcements in the Budget. So much of what we are doing is in capital (spending) in terms of building elective-only hospitals and building primary care centres,” he said.

Mr Harris was speaking on Monday after he and Taoiseach Leo Varadkar announced three changes to patient charges. The announcement was made at a new primary care centre in Grangegorman, Dublin 7.

The changes all arose from last October’s budget and comprise: a €10 reduction in the monthly drugs payment scheme threshold, bringing it down to €124; a 10 per cent increase in the weekly income thresholds for GP visit cards; as well as a reduction in prescription charges for medical card holders over 70 years of age. For the latter group, the new prescription charge will reduce from €2 to €1.50.
Costs

Mr Harris was responding to comments by the chairman of the Oireachtas Health Committee, Dr Michael Harty, earlier on Monday who said he was concerned that €20 million was not enough, given projected costs are €3 billion over the next six years of Sláintecare.

“It is delivering the new GP contract, which I hope the negotiations will conclude in the coming days. These are all parts of Sláintecare,” said Mr Harris.

“We have a lot to do. It is a 10-year plan. What we have seen in recent weeks is that plan come alive. The action plan published, the integration fund launched, the three announcements today, Laura Magahy and her team in place and doing a very good job,” he said.

Mr Harris said the message for people was if they were going to the pharmacy next month, their monthly bill was going to be cheaper. He said over 70s would get cheaper prescriptions and 44,000 families would benefit from a €10-a-month saving in the drug payment scheme threshold.

“This is actually the first increase in income thresholds for free GP care since 2006,” he said.
Brexit implications

Asked about the implications of Brexit for medicine supply in Ireland, he said there was no notification of any medicine shortage associated with Britain leaving the EU.

“The advice to patients doctors and everybody else remains the same: not to stockpile because if you stockpile you can inadvertently disrupt the supply chain.

“For most medications in Ireland there is about a 10-12 week supply here in the country,” he said.

Asked separately on his own views on Ireland rejoining the Commonwealth in the wake of Brexit, he said: “I don’t believe it arises. I don’t wish for it to arise. Indeed my party was the first party to declare a Republic and saw Ireland depart from such structures.

“I think there is an important point. We have to find new ways of working with the United Kingdom should they leave the EU. It’s healthy to have debates all these issues. It’s helpful to hear different views at part conferences. There are lots of different views in relation to it.

“We have a very strong, deep, historical relationship with the United Kingdom. We want to work out a way of making sure that continues. I don’t think that necessitates us remaining a member of the Commonwealth,” he said.

'''

article3 = '''
Media captionJussie Smollett: What Chicago Police say happened

Prosecutors have dropped all charges against US actor Jussie Smollett for allegedly staging a racist and homophobic attack, his lawyers say.

The Empire actor attended a court hearing in Chicago on Tuesday, where a judge reportedly sealed the case.

Mr Smollett pleaded not guilty this month to 16 counts of disorderly conduct for allegedly lying to police.

The black and gay actor thanked his supporters outside court after the hearing.

He said: "I have been truthful and consistent on every single level since day one.

"I would not be my mother's son if I was capable of one drop of what I've been accused of."

The Cook County State's Attorney's office said in a statement: "After reviewing all of the facts and circumstances of the case, including Mr Smollett's volunteer service in the community and agreement to forfeit his bond to the City of Chicago, we believe this outcome is a just disposition and appropriate resolution to this case", CBS News reported.

Smollett attorneys Tina Glandian and Patricia Brown Holmes said in a statement that their client's record "has been wiped clean."

"He was a victim who was villified and made to appear as a perpetrator as a result of false and inappropriate remarks made to the public causing an inappropriate rush to judgement," the lawyers said.

"Jussie and many others were hurt by these unfair and unwarranted actions. The entire situation is a reminder that there should never be an attempt to prove a case in the court of public opinion."

Mr Smollett said he was attacked in the early hours on 29 January in downtown Chicago outside a sandwich shop by two masked men who shouted racist and homophobic slurs, poured bleach on him and put a rope round his neck.

He claimed they shouted: "This is Maga country", a reference to President Donald Trump's "Make America Great Again" campaign slogan, and that at least one of the assailants was white.

But Chicago police said Mr Smollett, 36, paid two brothers, both of whom are black, $3,500 to stage the attack because he was "dissatisfied with his salary" on the TV show.

But his lawyer said outside court on Tuesday that the cheque was for nutrition and physical training.

Authorities also said Mr Smollett sent a racist and homophobic letter to himself at a Fox studio beforehand.

The US president lashed out last month at the actor on Twitter, saying, "what about MAGA and the tens of millions of people you insulted with your racist and dangerous comments!? #MAGA."

Mr Smollett's family said in a statement to US media on Tuesday that the actor had been vindicated.

"Jussie is a son, a brother, a partner, a champion for human rights, and a genuine soul who would never be capable of what he was falsely accused of," said their statement.

"He was the victim of an assault and then falsely blamed for his own attack. This morning truth has prevailed and he has been vindicated.

"All charges have been dismissed and his record expunged (cleared). The painful incidents we've witnessed him endure over the past 7 weeks have been heartbreaking and unjust to say the least.

"While many were quick to rush to judgement before hearing the actual truth, we are grateful that the truth about Jussie has come to light.

"We look forward to bringing the real perpetrators to justice. We thank God and our village for standing by us during this trying time." 
'''

In [78]:
wc = word_count(tweets)
wc[wc > 1]

great                           974
thank                           623
hillary                         502
people                          466
clinton                         320
trump                           305
big                             293
new                             290
many                            280
america                         280
country                         275
trump2016                       271
president                       248
today                           243
time                            213
u.s.                            203
going                           202
makeamericagreatagain           199
news                            187
bad                             178
good                            170
campaign                        168
trade                           164
jobs                            159
media                           157
cruz                            149
american                        147
being                       

716052